# Puzzle 1: N-body problem - total system energy

### 4 moons (Io, Europa, Ganymede, and Callisto); (x, y, z)-coordinates; gravity changes velocity +1 or -1 (+1 for lower value; -1 for higher value); new coordinates = old_coordinates + velocity; E_p = sum(coord); E_k = sum(vel); E = E_p * E_k

In [1]:
import numpy as np
from copy import deepcopy
from math import gcd

## Load input

In [2]:
with open('./input12.txt', 'r') as file:

    data = file.readlines()

#### some data preprocessing

In [3]:
data = [x.strip().strip('<').strip('>').split(', ') for x in data]

In [4]:
coordinate_list = np.zeros((len(data), len(data[0])))
for i, planet in enumerate(data):
    for j, coordinate in enumerate(planet):
        for k, char in enumerate(coordinate):
            if char in ('x', 'y', 'z'):
                coordinate_list[i][j] = coordinate[k+2:]
                break

In [5]:
class moon_simulator(object):
    
    def __init__(self, coordinate_list):
        
        self.coordinate_list = coordinate_list
        self.velocity_list = np.zeros(coordinate_list.shape)
        self.number_planets = len(coordinate_list)
        
    def step(self):
        
        for i, coordinates in enumerate(self.coordinate_list):
            smaller = (coordinates < np.delete(self.coordinate_list, i, 0)).astype(int)
            equal = (coordinates == np.delete(self.coordinate_list, i, 0)).astype(int)
            # mapping: y/(num_pl-1) + 2*(num_pl-1) - (num_pl-1)
            delta_vel = sum(smaller)*2-(self.number_planets-1)+sum(equal)
            self.velocity_list[i] += delta_vel
            
        self.coordinate_list += self.velocity_list
        
    def energy(self):
        
        self.e_p = np.sum(abs(self.coordinate_list), 1)
        self.e_k = np.sum(abs(self.velocity_list), 1)
        self.e = np.sum(self.e_p*self.e_k)

In [6]:
steps = 1000
moons = moon_simulator(deepcopy(coordinate_list))
for i in range(0, steps):
    moons.step()

moons.energy()
print('The energy of the system after {0} steps is {1}.'.format(steps, moons.e))


The energy of the system after 1000 steps is 7179.0.


# Puzzle 2: N-body problem - calculate length of period

### all the axes are independent, so we can split the system; whole period is the least common multiple of all three periods

#### at first, set up a moon simulator for each axis and store the initial state

In [7]:
simulator_dict = dict()
states_init_dict = dict()
for i in range(0, len(coordinate_list[0])):
    simulator_dict['{0}'.format(i)] = moon_simulator(deepcopy(coordinate_list[:, i].reshape(-1, 1)))
    states_init_dict['{0}'.format(i)] = np.append(simulator_dict['{0}'.format(i)].coordinate_list,
                                                 simulator_dict['{0}'.format(i)].velocity_list)

#### now, simulate the systems until they reach their initial state and calculate the lcm

In [8]:
def lcm(values):
    multiple = values[0]
    for i in range(1, len(values)):
        multiple = multiple*values[i] // gcd(multiple, values[i])
    return multiple

In [9]:
periods = []
for key in simulator_dict.keys():
    state = np.zeros(states_init_dict[key].shape)
    period = 0
    while sum(states_init_dict[key] == state) != len(states_init_dict[key]):
        simulator_dict[key].step()
        state = np.append(simulator_dict[key].coordinate_list, simulator_dict[key].velocity_list)
        period += 1
    periods.append(period)
print('The period length of the system is {0}.'.format(lcm(periods)))

The period length of the system is 428576638953552.
